In [1]:
import string
import re
from os import listdir
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [3]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    
    return tokens

In [4]:
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [5]:
# load all docs in a directory
def process_docs(directory, vocab):
    lines = list()
    # walk through all files in the folder
    for filename in listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        line = doc_to_line(path, vocab) 
        # add to list
        lines.append(line)
    return lines

In [6]:
# load and clean a dataset
def load_clean_dataset(vocab):
    # load documents
    neg = process_docs(r'c:/Download/review_polarity/txt_sentoken/neg',vocab)
    pos = process_docs(r'c:/Download/review_polarity/txt_sentoken/pos',vocab)
    docs = neg + pos
    # prepare labels
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    return docs, labels

In [7]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [8]:
# define the model
def define_model(n_words):
    # define network
    model = Sequential()
    model.add(Dense(50, input_shape=(n_words,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile network
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [9]:
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, model):
    # clean
    tokens = clean_doc(review)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    # convert to line
    line = ' '.join(tokens)
    # encode
    encoded = tokenizer.texts_to_matrix([line], mode='binary')
    # predict sentiment
    yhat = model.predict(encoded, verbose=0)
    print("yhat :", yhat)
    # retrieve predicted percentage and label
    percent_pos = yhat[0,0]
    print("percent_pos :" ,percent_pos)
    if round(percent_pos) == 0:
        return (1-percent_pos), 'NEGATIVE'
    return percent_pos, 'POSITIVE'

In [10]:
# load the vocabulary
vocab_filename = 'newvocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())


In [11]:
# load all reviews
train_docs, ytrain = load_clean_dataset(vocab)
test_docs, ytest = load_clean_dataset(vocab)


In [12]:
import numpy as np

In [13]:
ytrain = np.array(ytrain)

In [14]:
ytest = np.array(ytest)

In [15]:
# create the tokenizer
tokenizer = create_tokenizer(train_docs)


In [16]:

# encode data
Xtrain = tokenizer.texts_to_matrix(train_docs, mode='binary')
Xtest = tokenizer.texts_to_matrix(test_docs, mode='binary')


In [17]:
# define network
n_words = Xtrain.shape[1]
print("n_words : ", n_words)
model = define_model(n_words)


n_words :  25768
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1288450   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 1,288,501
Trainable params: 1,288,501
Non-trainable params: 0
_________________________________________________________________


In [18]:

# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)


Epoch 1/10
63/63 - 0s - loss: 0.4773 - accuracy: 0.7765
Epoch 2/10
63/63 - 0s - loss: 0.0738 - accuracy: 0.9895
Epoch 3/10
63/63 - 0s - loss: 0.0170 - accuracy: 1.0000
Epoch 4/10
63/63 - 0s - loss: 0.0061 - accuracy: 1.0000
Epoch 5/10
63/63 - 0s - loss: 0.0031 - accuracy: 1.0000
Epoch 6/10
63/63 - 0s - loss: 0.0019 - accuracy: 1.0000
Epoch 7/10
63/63 - 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 8/10
63/63 - 0s - loss: 8.7687e-04 - accuracy: 1.0000
Epoch 9/10
63/63 - 0s - loss: 6.1168e-04 - accuracy: 1.0000
Epoch 10/10
63/63 - 0s - loss: 4.4050e-04 - accuracy: 1.0000


In [19]:
# test positive text
text = 'Best movie ever! It was great, I recommend it.'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))


yhat : [[0.55669147]]
percent_pos : 0.55669147
Review: [Best movie ever! It was great, I recommend it.]
Sentiment: POSITIVE (55.669%)


In [20]:
# test negative text
text = 'This is a bad movie.'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))

yhat : [[0.36404124]]
percent_pos : 0.36404124
Review: [This is a bad movie.]
Sentiment: NEGATIVE (63.596%)
